Step: Load data and packages

In [3]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [15]:
import pandas as pd
import pyodbc
import numpy as np
from skimpy import clean_columns
from pyprojroot import here
import os
import PyPDF2
import numpy as np
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import docx

from PyPDF2 import PdfReader

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np

nltk.download("punkt")
nltk.download("stopwords")
path_data = here("./data")
os.chdir(path_data)

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Step: Combine text by case id, and lower text

In [17]:
def read_word_file(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text
    return text

In [18]:
cv_path = "./CV_Matt_Hanauer.docx"
job_description_path = "./job_example.docx"

cv_text = read_word_file(cv_path)
job_description_text = read_word_file(job_description_path)

In [19]:
def preprocess_text(text):
    words = word_tokenize(text.lower())
    words_filtered = [word for word in words if word.isalnum()]
    return words_filtered


def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


def compare_texts(text1_tokens, text2_tokens, top_n_scores=10):
    text1_sentences = " ".join(text1_tokens)
    text2_sentences = " ".join(text2_tokens)

    text1_embedding = model.encode(text1_sentences, convert_to_tensor=True)
    text2_embedding = model.encode(text2_sentences, convert_to_tensor=True)

    similarity_score = cosine_similarity(text1_embedding, text2_embedding).round(2)
    return similarity_score

In [20]:
compare_texts(cv_text, job_description_text)

0.93

Step: Identify which key words are missing

In [21]:
from nltk.corpus import stopwords
import nltk
from nltk.probability import FreqDist

# Load the English stop words
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


def calculate_word_frequency(text_tokens):
    filtered_tokens = [token for token in text_tokens if token not in stop_words]
    fdist = FreqDist(filtered_tokens)
    return fdist

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Preprocess the texts
resume_tokens = preprocess_text(cv_text)
job_description_tokens = preprocess_text(job_description_text)

In [23]:
# Calculate frequency distributions
resume_fdist = calculate_word_frequency(resume_tokens)
job_description_fdist = calculate_word_frequency(job_description_tokens)

In [24]:
# Get the top 10 words from the job description
top_n = 10
top_job_description_words = [
    word for word, freq in job_description_fdist.most_common(top_n)
]

# Find the missing top 10 words in the resume
missing_keywords = []
for word in top_job_description_words:
    if word not in resume_fdist:
        missing_keywords.append(word)

missing_keywords

['technical', 'availity', 'years', 'cloud', 'production']